In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, metrics

import datetime as dt
from datetime import datetime, timedelta

sns.set_style('darkgrid')
pd.options.display.float_format = '{:,.3f}'.format

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Warnings

from warnings import filterwarnings
filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Data Preprocessing

from sklearn.neighbors import LocalOutlierFactor 
from sklearn import preprocessing

# Modeling

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from xgboost import XGBRegressor
!pip install lightgbm
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Model Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

## Load data

In [ ]:
%%time

parser = lambda date: pd.to_datetime(date, format='%d.%m.%Y')

train = pd.read_csv('../input/sales_train.csv', parse_dates=['date'], date_parser=parser)


In [ ]:
df=train.copy()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["date"].max()

### Date değişkeninden farklı bilgiler çıkarmak mümkündür. Eğer içinde gün-ay-yıl geçiyorsa buradan tarihle alakalı bir çok bilgiye erişilebilir. Örneğin; bu date değişkenin gün, ay, yıl, hangi gün, haftanın günü gibi tarihle alakalı bir çok değişken yaratılabilir.


In [ ]:
#date kolononu mutlaka datetime yapmak gereklidir.

df["date"]=pd.to_datetime(df["date"], format='%Y%m%d', errors='ignore')

In [ ]:
def create_date_features(df):
    df['month'] = df.date.dt.month 
    df['day_of_month'] = df.date.dt.day
    df['day_of_year'] = df.date.dt.dayofyear
    df['year'] = df.date.dt.year
    df["is_wknd"] = df.date.dt.weekday // 4
    df['is_month_start'] = df.date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.date.dt.is_month_end.astype(int)
    df["DAY_OF_WEEKDAY"]=df.date.dt.dayofweek
    return df

In [ ]:
df = create_date_features(df)

In [ ]:
df.head()

In [ ]:
df["year"].value_counts()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

# Aykırı değer analizi

In [ ]:
sns.set(style="whitegrid")

sns.boxplot(df["item_price"])

In [ ]:
sns.kdeplot(df["item_price"], shade=True)

In [ ]:
df["item_price"].describe()

In [ ]:
# satışta negatif değerler çıktığı için filtreleme yaparak sadece pozitif değerler alalım.

df=df.loc[df["item_price"]>0]

In [ ]:
Q1=df["item_price"].quantile(0.01)
Q3=df["item_price"].quantile(0.99)

IQR= Q3 - Q1
IQR

maksimum=Q3 + 1.5 * IQR
maksimum

In [ ]:
# outlier tespiti
hepsi= (df["item_price"]>maksimum)  #filtre

df["item_price"][hepsi].count() #aykırı gözlemlere erişme

In [ ]:
#outlier sadece sağ taraftakileri silme - 10091 tane silme

df=df.drop((df["item_price"][(df["item_price"]>maksimum)]).index)

In [ ]:
df.shape

### KOrelasyon

In [ ]:
corr = df.corr()
plt.figure(figsize=(18,10))
sns.heatmap(corr, annot=True)
plt.show()

# test train split

In [ ]:
#tarihe göre küçükte büyüğe doğru sıralama
df= df.sort_values(by=["date"])

In [ ]:
df.drop(["date"], inplace=True, axis=1)

In [ ]:
df["year"].value_counts()

In [ ]:
609770 / 2700000

# 2013 ve 2014 train
# 2015 test olarak alındı
#yaklaşık olarak %22 si test oldu

In [ ]:
Train = df[df["year"].isin([2013, 2014])]
Test = df[df["year"].isin([2015])]

In [ ]:
X_train = Train.drop(columns=['item_price',"year"], axis=1)
y_train = Train["item_price"]

In [ ]:
X_test = Test.drop(columns=['item_price',"year"], axis=1)
y_test = Test["item_price"]

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# # model kurma LightGBM

In [ ]:
import lightgbm as lgb

#categorical_feature içine liste şeklinde kategorik olan değişkenleri veriyoruz.

d_train=lgb.Dataset(X_train,label=y_train, categorical_feature=['shop_id', 'item_id'], free_raw_data=False)

In [ ]:
lgbm_params={ 'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['mse'],
    }

In [ ]:
clf=lgb.train(lgbm_params,d_train,100)

In [ ]:
y_pred=clf.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, y_pred)

# CatBoost

Catboost modeli kategorik verilerle de çalışabilir. Ancak cat_features parametresi altında verdiğimiz değişkenlerin "object veya category" tipinde olması gereklidir. Bu nedenle değişkenler "item_id" ile "shop_id" object tipine çevrilip modele konulmuştur.

In [ ]:
df1=df.copy()

In [ ]:
#object haline getirildi. modellerin çalışması için

df1["item_id"]=df1["item_id"].astype(object)
df1["shop_id"]=df1["shop_id"].astype(object)

In [ ]:
Train = df1[df1["year"].isin([2013, 2014])]
Test = df1[df1["year"].isin([2015])]

#test train ayrılması
X_train = Train.drop(columns=['item_price',"year"], axis=1)
y_train = Train["item_price"]

X_test = Test.drop(columns=['item_price',"year"], axis=1)
y_test = Test["item_price"]

In [ ]:
cat_features=['item_id',"shop_id"]

In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool
from catboost import MetricVisualizer
import catboost as cbd

train_dataset = cbd.Pool(X_train, y_train, cat_features = cat_features) 
test_dataset = cbd.Pool(X_test, y_test, cat_features = cat_features)

In [ ]:
model = cbd.CatBoostRegressor(loss_function="RMSE", n_estimators=20 )

model.fit(train_dataset, verbose=0, plot=False)

#verinin boyutu fazla olduğu için subsample gibi n_estimators değeri 20 olarak alındı. Yani ağaç daha az çalıştırıldı.
#böylece hızlı bir şekilde sonuç aldık.
#sonra model tüm veri ile tekrardan çalıştırılacak.

#ayrıca bir çözüm yolu olarak n_estimators parametresi yerine iterations=100 alınarak hızlı bir şekilde sonuç alınır.

In [ ]:
#train skoru
pred_train = model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, pred_train)))
rmse

In [ ]:
#test skoru
pred = model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
rmse

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test, pred)

### ayrıca 3. çözüm yolu olarak subsample alınabilir. Train üzerinden seçilen daha az  veri ile model kurulup sonuçlara bakılabilir.

sam_train = Train.sample(10000, random_state=0)

X_train = sam_train.drop(columns=['item_price',"year"], axis=1)

y_train = sam_train["item_price"]

yazıldıktan sonra tekrar split edilerek model kurulur.